# Applying Indicators to Price Data
Takes the output from `iq_feed_cleaning.ipynb` and for each row, adds indicators. \
This is preferred over calculating the indicators for each timestamp/date, as it saves 
a lot of computations. \
It does require us to do a look-up, but so does the alternative.

In [ ]:
# Sort by stocks
# Sort by time
# Add stock specific indicator
# Add global indicator (S&P500)

In [2]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import plotly.express as px
import pytz
eastern = pytz.timezone('US/Eastern')
from src.config import config
from concurrent.futures import ThreadPoolExecutor
import os

In [ ]:
onlyfiles = [f for f in listdir(config.data.iqfeed.daily.cleaned) if isfile(join(config.data.iqfeed.daily.cleaned, f))]
tickers = [x.split("_")[0] for x in onlyfiles]
tickers = np.sort(tickers)

In [8]:
def add_indicators(ticker):
    prices = pd.read_parquet(path=f"{config.data.iqfeed.daily.cleaned}/{ticker}_daily.parquet")
    prices["std_252"] = prices["adj_close"].pct_change().rolling(252, min_periods=252).std()*(252**0.5)
    prices["dollar_volume"] = prices["adj_volume"] * (prices["adj_close"] + prices["adj_open"])/2
    prices["r_intra_(t-1)"] = (prices["adj_close"] / prices["adj_open"]).shift(periods=1)
    prices.to_parquet(path=f"{config.data.iqfeed.daily.cleaned}/{ticker}_daily.parquet")

TypeError: NDFrame.rolling() got an unexpected keyword argument 'min_period'

In [ ]:
pool_obj = ThreadPoolExecutor(max_workers=os.cpu_count()-1)
ans = pool_obj.map(add_indicators, tickers)
result = list(ans)